In [16]:
using ModelingToolkit, DomainSets, Sophon, CairoMakie, Lux
using Optimization, OptimizationOptimJL

@parameters x, t
@variables θ₁(..), θ₂(..), w₁(..), w₂(..)
Dₜ = Differential(t)
Dₜ² = Differential(t)^2
Dₓ = Differential(x)
Dₓ² = Differential(x)^2

f₁(x,t) = cos(t)*(1-sin(x))
f₂(x,t) = 2/π * cos(t) - π/2 * sin(x) * cos(t)

eqs = [
    Dₓ(θ₁(x,t)) - Dₓ²(w₁(x,t)) + Dₜ²(w₁(x,t)) + w₁(x,t) - w₂(x,t) ~ f₁(x,t),
    Dₓ(θ₂(x,t)) - Dₓ²(w₂(x,t)) + Dₜ²(w₂(x,t)) + w₂(x,t) - w₁(x,t) ~ f₂(x,t),
    Dₜ²(θ₁(x,t)) ~ Dₓ²(θ₁(x,t)) + Dₓ(w₁(x,t)) - θ₁(x,t),
    Dₜ²(θ₂(x,t)) ~ Dₓ²(θ₂(x,t)) + Dₓ(w₂(x,t)) - θ₂(x,t),
]

bcs = [
    θ₁(x, 0.0) ~ π/2*cos(x) + x - π/2,
    Dₜ(θ₁(x, 0.0)) ~ 0.0,
    w₁(x, 0.0) ~ π/2*sin(x),
    Dₜ(w₁(x, 0.0)) ~ 0.0,
    θ₂(x, 0.0) ~ cos(x) + 2/π * x - 1,
    Dₜ(θ₂(x, 0.0)) ~ 0.0,
    w₂(x, 0.0) ~ sin(x),
    Dₜ(w₂(x, 0.0)) ~ 0.0,
    θ₁(0.0, t) ~ 0.0,
    θ₁(π, t) ~ 0.0,
    w₁(0.0, t) ~ 0.0,
    w₁(π, t) ~ 0.0,
    θ₂(0.0, t) ~ 0.0,
    θ₂(π, t) ~ 0.0,
    w₂(0.0, t) ~ 0.0,
    w₂(π, t) ~ 0.0]

    
domains = [x ∈ Interval(0.0, π),
           t ∈ Interval(0.0, 1.0)]

@named timo_double = PDESystem(eqs, bcs, domains, [x,t], [θ₁,θ₂,w₁,w₂])

PDESystem
Equations: Equation[Differential(t)(Differential(t)(w₁(x, t))) + Differential(x)(θ₁(x, t)) + w₁(x, t) - Differential(x)(Differential(x)(w₁(x, t))) - w₂(x, t) ~ (1 - sin(x))*cos(t), Differential(t)(Differential(t)(w₂(x, t))) + Differential(x)(θ₂(x, t)) + w₂(x, t) - Differential(x)(Differential(x)(w₂(x, t))) - w₁(x, t) ~ 0.6366197723675814cos(t) - 1.5707963267948966cos(t)*sin(x), Differential(t)(Differential(t)(θ₁(x, t))) ~ Differential(x)(Differential(x)(θ₁(x, t))) + Differential(x)(w₁(x, t)) - θ₁(x, t), Differential(t)(Differential(t)(θ₂(x, t))) ~ Differential(x)(Differential(x)(θ₂(x, t))) + Differential(x)(w₂(x, t)) - θ₂(x, t)]
Boundary Conditions: Equation[θ₁(x, 0.0) ~ x + 1.5707963267948966cos(x) - 1.5707963267948966, Differential(t)(θ₁(x, 0.0)) ~ 0.0, w₁(x, 0.0) ~ 1.5707963267948966sin(x), Differential(t)(w₁(x, 0.0)) ~ 0.0, θ₂(x, 0.0) ~ 0.6366197723675814x + cos(x) - 1, Differential(t)(θ₂(x, 0.0)) ~ 0.0, w₂(x, 0.0) ~ sin(x), Differential(t)(w₂(x, 0.0)) ~ 0.0, θ₁(0.0, t) ~

In [17]:
pinn = PINN(θ₁ = Siren(2,1; hidden_dims=16,num_layers=4,omega=1),
            θ₂ = Siren(2,1; hidden_dims=16,num_layers=4,omega=1),
            w₁ = Siren(2,1; hidden_dims=16,num_layers=4,omega=1),
            w₂ = Siren(2,1; hidden_dims=16,num_layers=4,omega=1))

sampler = QuasiRandomSampler(2000, 500)
strategy = NonAdaptiveTraining(1, 10)

prob = Sophon.discretize(timo_double, pinn, sampler, strategy)

OptimizationProblem. In-place: true
u0: ComponentVector{Float64}(θ₁ = (layer_1 = (weight = [-0.40679049491882324 -0.06941556930541992; -0.033619701862335205 0.19977039098739624; … ; 0.23553550243377686 -0.48516666889190674; -0.10317927598953247 0.3670007586479187], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = [0.26913562417030334 0.24725890159606934 … 0.0751800537109375 -0.3486098349094391; -0.24230173230171204 0.4124480187892914 … 0.44783326983451843 -0.2522532343864441; … ; -0.2407858669757843 0.35609203577041626 … 0.4278322160243988 0.33957594633102417; 0.3919776678085327 -0.12697243690490723 … 0.4084819257259369 -0.02673591487109661], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = [-0.06881769746541977 0.45272815227508545 … 0.0744585171341896 -0.12980814278125763; 0.5762389302253723 -0.5881596803665161 … 0.5079026818275452 -0.2438454031944275; … ; 0.6092303395271301 -0.5541239976882935 … -0.16043752431869507 0.14571909606456757; -0.45503365993499756 -0.186248168

In [18]:

iter = ProgressBar(1:1000)

function callback(p, l)
    set_description(iter, string(@sprintf("Loss: %.2f", l)))
    return false
end

bfgs = BFGS()

@time res = Optimization.solve(prob, bfgs; maxiters=1000, callback=callback)

Loss: 100.21799578910638


Loss: 37.51771066000177


Loss: 19.450019355481825


Loss: 14.313577741151546


Loss: 11.866450899867784


...

Loss: 1.1811156283807815e-6


Loss: 1.1787401256828352e-6


Loss: 1.1766642858555321e-6


Loss: 1.1740796778231657e-6
565.973000 seconds (87.97 M allocations: 1.089 TiB, 3.43% gc time)


u: ComponentVector{Float64}(θ₁ = (layer_1 = (weight = [-0.09453200801722574 -0.05049845651140615; -0.16643005586818138 0.3512534649203298; … ; 0.3129608527850399 -0.2570967149907712; -0.04234293684893176 0.11905989729438705], bias = [-0.14301900786013969; -0.030073321402754746; … ; -0.16637374062173826; -0.0005580326198599759;;]), layer_2 = (weight = [0.380766294572286 0.32520370093814155 … -0.10227198036309545 -0.3462807302082656; -0.18853711434207338 0.3757782856283073 … 0.43781317634089134 -0.3023513608579294; … ; -0.15333141567935835 0.3204666263997438 … 0.5666600637753831 0.2530108397075421; 0.35498589783345486 -0.08686113184229047 … 0.35170086509390597 0.004742733450935375], bias = [-0.144869316475206; -0.08198747225185869; … ; -0.009084976556300573; 0.19214974994083925;;]), layer_3 = (weight = [-0.12567038406860995 0.3733446311048544 … 0.09831548888476088 -0.10650225642547878; 0.8043098760916081 -0.5241285304325273 … 0.4348833152327801 -0.1531561446426654; … ; 0.6235944887672962

In [28]:
res.objective

1.1740796778231657e-6

In [33]:
prob = remake(prob;u0=res.u)
@time res = Optimization.solve(prob, bfgs; maxiters=1000)

In [ ]:
res.objective

In [ ]:
phi = pinn.phi
ps = res.u

xs, ts= [infimum(d.domain):0.01:supremum(d.domain) for d in timo_double.domain]


θ₁ = [sum(phi.θ₁(([x,t]), ps.θ₁)) for x in xs, t in ts]
θ₂ = [sum(phi.θ₂(([x,t]), ps.θ₂)) for x in xs, t in ts]
w₁ = [sum(phi.w₁(([x,t]), ps.w₁)) for x in xs, t in ts]
w₂ = [sum(phi.w₂(([x,t]), ps.w₂)) for x in xs, t in ts]

fig = Figure(resolution=(1200,1200))
axis = (xlabel="x", ylabel="t", title="f1")


_, hm3 = heatmap(fig[1, 1], xs, ts, θ₁, axis= merge(axis, (; title="θ1")), colormap=:jet)
Colorbar(fig[1, 2], hm3)
_, hm4 = heatmap(fig[1, 3], xs, ts, θ₂, axis= merge(axis, (; title="θ2")), colormap=:jet)
Colorbar(fig[1, 4], hm4)
_, hm5 = heatmap(fig[2, 1], xs, ts, w₁, axis= merge(axis, (; title="w1")), colormap=:jet)
Colorbar(fig[2, 2], hm5)
_, hm6 = heatmap(fig[2, 3], xs, ts, w₂, axis= merge(axis, (; title="w2")), colormap=:jet)
Colorbar(fig[2, 4], hm6)

display(fig)

In [21]:
θ₁_analytic(x,t) = (π/2*cos(x) + x - π/2)*cos(t) 
θ₂_analytic(x,t) = (cos(x) + 2/π * x - 1)*cos(t)
w₁_analytic(x,t) = π/2*sin(x)*cos(t)
w₂_analytic(x,t) = sin(x)*cos(t)

w₂_analytic (generic function with 1 method)

In [ ]:
θ₁_true = [θ₁_analytic(x,t) for x in xs, t in ts]
θ₂_true = [θ₂_analytic(x,t) for x in xs, t in ts]
w₁_true = [w₁_analytic(x,t) for x in xs, t in ts]
w₂_true = [w₂_analytic(x,t) for x in xs, t in ts]

fig = Figure(resolution=(1200,1200))
axis = (xlabel="x", ylabel="t", title="f1 error")


_, hm3 = heatmap(fig[1, 1], xs, ts, abs.(θ₁ .- θ₁_true), axis= merge(axis, (; title="θ1")), colormap=:jet)
Colorbar(fig[1, 2], hm3)
_, hm4 = heatmap(fig[1, 3], xs, ts, abs.(θ₂ .- θ₂_true), axis= merge(axis, (; title="θ2")), colormap=:jet)
Colorbar(fig[1, 4], hm4)

_, hm5 = heatmap(fig[2, 1], xs, ts, abs.(w₁ .- w₁_true), axis= merge(axis, (; title="w1")), colormap=:jet)
Colorbar(fig[2, 2], hm5)
_, hm6 = heatmap(fig[2, 3], xs, ts, abs.(w₂ .- w₂_true), axis= merge(axis, (; title="w2")), colormap=:jet)
Colorbar(fig[2, 4], hm6)


display(fig)